# Imports / Setup

In [409]:
import pandas as pd
# pd.options.display.max_columns = 50
# pd.options.display.max_colwidth = 150
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate,\
                                    GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

import nltk
from nltk import pos_tag
from nltk.corpus import wordnet, stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
sw = stopwords.words('english')

# Helper Functions/Code

Courtesy of Flatiron's lecture materials, my colleague Saad, and various others - credit due to Daniel on helping me figure out the regex pattern to exclude Twitter mentions.

In [410]:
def prepare_doc(doc, stem=True):
    
    # Establish regex pattern, instantiate tokenizer
    # The regex pattern also excludes @mentions utilizing a
    # negative lookbehind
    regex_token = RegexpTokenizer(r"(?<![@A-Za-z0-9_])([a-zA-Z]+(?:’[a-z]+)?)")
    # Create tokens for doc
    doc = regex_token.tokenize(doc)
    # Lowercase all words
    doc = [word.lower() for word in doc]
    # Remove stopwords
    doc = [word for word in doc if word not in sw]
    # Stem!
    stemmer = SnowballStemmer("english")
    if stem:
        doc = [stemmer.stem(word) for word in doc]
    return ' '.join(doc)


def cv_printScores(cv_metric):
    print('CV Results')
    print('='*32)
    print('Accuracy')
    print('-'*32)
    print(f"Training accuracy: {cv_metric['train_accuracy'].mean():.3f}")
    print(f"Test accuracy:     {cv_metric['test_accuracy'].mean():.3f}\n")
    print('F-1 Score')
    print('-'*32)
    print(f"Training F1 score: {cv_metric['train_f1_macro'].mean():.3f}")
    print(f"Test F1 score:     {cv_metric['test_f1_macro'].mean():.3f}")

# Data: Loading, Cleaning, and Munging

In [411]:
df = pd.read_csv('../../data/judge_1377884607_tweet_product_company.csv')

In [412]:
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
8716,Ipad everywhere. #SXSW {link},iPad,Positive emotion
8717,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
8718,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
8719,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


This line of code is used to print an example tweet from the dataset; this was helpful in creating a functional regular expression pattern using outside web tools.

In [413]:
df.iloc[666].tweet_text

'I won the iPad accessories bag! Thanks @mention #SXSW'

Let's rename the columns for ease of use. Note that `product` is a reserved word in Python; for the second column, initially titled `emotion_in_tweet_is_directed_at`, I use `product_name`.

In [414]:
df.rename(columns = {'tweet_text': 'text',
                     'emotion_in_tweet_is_directed_at': 'product_name',
                     'is_there_an_emotion_directed_at_a_brand_or_product': 'emotion'},
          inplace = True)

Checking for nulls...

In [415]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8721 entries, 0 to 8720
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text          8720 non-null   object
 1   product_name  3169 non-null   object
 2   emotion       8721 non-null   object
dtypes: object(3)
memory usage: 204.5+ KB


Lots of nulls in `product_name`, which we'll have to deal with at one point or another, and exactly *one* null value in the `text` column. Let's check that out real quick...

In [416]:
df.loc[df['text'].isna()]

,text,product_name,emotion
6,NaN,NaN,No emotion toward brand or product


Totally empty. We can drop that.

In [417]:
df.dropna(subset=['text'],
          inplace=True)

## Distribution of target values

Let's get a look of how the target column (`emotion`) and the third column, `product_name`, are distributed.

In [418]:
print(f"Product name value counts:\n{df.product_name.value_counts()}")
print('-' * 32)
print(f"Emotion value counts:\n{df.emotion.value_counts()}")
print('-' * 32)
print(f"Emotion value counts (normalized):\n{df.emotion.value_counts(normalize=True)}")

Product name value counts:
iPad                               910
Apple                              640
iPad or iPhone App                 451
Google                             412
iPhone                             288
Other Google product or service    282
Android App                         78
Android                             74
Other Apple product or service      34
Name: product_name, dtype: int64
--------------------------------
Emotion value counts:
No emotion toward brand or product    5155
Positive emotion                      2869
Negative emotion                       545
I can't tell                           151
Name: emotion, dtype: int64
--------------------------------
Emotion value counts (normalized):
No emotion toward brand or product    0.591170
Positive emotion                      0.329014
Negative emotion                      0.062500
I can't tell                          0.017317
Name: emotion, dtype: float64


`I can't tell` makes up less than 2% of our dataset, and doesn't offer much more information in the way of word significance than the tweets labeled `No emotion toward brand or product`. It might be worth dropping records where `emotion` has been recorded as `I can't tell`.

In [419]:
df.drop(
    index = df[df.emotion == "I can't tell"].index,
    inplace = True)

df.head()

,text,product_name,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [420]:
sample1 = df.iloc[10].text

In [421]:
# Testing the custom function from up top
prepare_doc(doc=sample1,
            stem=True)

'find amp start impromptu parti sxsw http bit ly gvlrin wait til android app come'

## Stemming

In [422]:
full_text = df['text'].apply(lambda x: prepare_doc(x, stem=True))

full_text

0       iphon hrs tweet rise dead need upgrad plugin s...
1       know awesom ipad iphon app like appreci design...
2                                wait ipad also sale sxsw
3             hope year festiv crashi year iphon app sxsw
4       great stuff fri sxsw marissa mayer googl tim r...
                              ...                        
8716                             ipad everywher sxsw link
8717    wave buzz rt interrupt regular schedul sxsw ge...
8718    googl zeiger physician never report potenti ae...
8719    verizon iphon custom complain time fell back h...
8720                  rt googl test check offer sxsw link
Name: text, Length: 8569, dtype: object

In [423]:
df['tokens'] = full_text

df.head()

,text,product_name,emotion,tokens
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,iphon hrs tweet rise dead need upgrad plugin s...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,know awesom ipad iphon app like appreci design...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,wait ipad also sale sxsw
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,hope year festiv crashi year iphon app sxsw
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,great stuff fri sxsw marissa mayer googl tim r...


In [424]:
df['text'][:20]

0     .@wesley83 I have a 3G iPhone. After 3 hrs twe...
1     @jessedee Know about @fludapp ? Awesome iPad/i...
2     @swonderlin Can not wait for #iPad 2 also. The...
3     @sxsw I hope this year's festival isn't as cra...
4     @sxtxstate great stuff on Fri #SXSW: Marissa M...
5     @teachntech00 New iPad Apps For #SpeechTherapy...
7     #SXSW is just starting, #CTIA is around the co...
8     Beautifully smart and simple idea RT @madebyma...
9     Counting down the days to #sxsw plus strong Ca...
10    Excited to meet the @samsungmobileus at #sxsw ...
11    Find &amp; Start Impromptu Parties at #SXSW Wi...
12    Foursquare ups the game, just in time for #SXS...
13    Gotta love this #SXSW Google Calendar featurin...
14    Great #sxsw ipad app from @madebymany: http://...
15    haha, awesomely rad iPad app by @madebymany ht...
16    Holler Gram for iPad on the iTunes App Store -...
17    I just noticed DST is coming this weekend. How...
18    Just added my #SXSW flights to @planely. M

In [425]:
df['tokens'][:20]

0     iphon hrs tweet rise dead need upgrad plugin s...
1     know awesom ipad iphon app like appreci design...
2                              wait ipad also sale sxsw
3           hope year festiv crashi year iphon app sxsw
4     great stuff fri sxsw marissa mayer googl tim r...
5     new ipad app speechtherapi communic showcas sx...
7     sxsw start ctia around corner googleio hop ski...
8     beauti smart simpl idea rt wrote hollergram ip...
9     count day sxsw plus strong canadian dollar mea...
10    excit meet sxsw show sprint galaxi still run a...
11    find amp start impromptu parti sxsw http bit l...
12    foursquar up game time sxsw http j mp grn stil...
13    gotta love sxsw googl calendar featur top part...
14                 great sxsw ipad app http tinyurl com
15    haha awesom rad ipad app http bit ly htdfim ho...
16    holler gram ipad itun app store http co kfn vi...
17    notic dst come weekend mani iphon user hour la...
18    ad sxsw flight match peopl plane airport a

In [426]:
word_freq = FreqDist()

for tweet in df['text'].map(lambda x: prepare_doc(x, stem = False)):
    for word in tweet.split():
        word_freq[word] += 1
        
word_freq.most_common(n = 50)

[('sxsw', 9108),
 ('link', 4077),
 ('rt', 2925),
 ('ipad', 2848),
 ('google', 2504),
 ('apple', 2184),
 ('quot', 1582),
 ('iphone', 1497),
 ('store', 1397),
 ('new', 1057),
 ('austin', 919),
 ('amp', 803),
 ('app', 791),
 ('circles', 639),
 ('social', 632),
 ('launch', 628),
 ('today', 566),
 ('android', 563),
 ('pop', 543),
 ('network', 447),
 ('via', 400),
 ('line', 391),
 ('get', 383),
 ('free', 378),
 ('called', 353),
 ('mobile', 342),
 ('party', 335),
 ('sxswi', 333),
 ('major', 301),
 ('one', 297),
 ('like', 275),
 ('w', 260),
 ('time', 259),
 ('check', 257),
 ('temporary', 254),
 ('opening', 242),
 ('possibly', 240),
 ('day', 228),
 ('people', 223),
 ('see', 217),
 ('downtown', 216),
 ('mayer', 212),
 ('great', 211),
 ('going', 211),
 ('maps', 211),
 ('apps', 209),
 ('go', 201),
 ('popup', 198),
 ('need', 196),
 ('open', 193)]

Looking at the total value counts, we can identify some other tokens/words that might be worth adding to our stopwords list:
- `sxsw` & `sxswi` / `austin` - tokens that refer to setting - all tweets in corpus seem to be pulled from a South by Southwest (SXSW) festival event.
- `rt` / `link` / `quot` / `mobile` - tokens that refer to Twitter interactions or metadata about the post itself; not semanticaly useful information
- `amp` - ampersand (&) character that was improperly transcoded

In [427]:
sw.extend([
    'sxsw',
    'sxswi',
    'austin',
    'rt',
    'quot',
    'mention',
    'link',
    'amp',
    'mobil'
])

In [428]:
le = LabelEncoder()

df['target'] = le.fit_transform(df['emotion'])

le.classes_

array(['Negative emotion', 'No emotion toward brand or product',
       'Positive emotion'], dtype=object)

In [429]:
df

,text,product_name,emotion,tokens,target
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,iphon hrs tweet rise dead need upgrad plugin s...,0
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,know awesom ipad iphon app like appreci design...,2
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,wait ipad also sale sxsw,2
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,hope year festiv crashi year iphon app sxsw,0
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,great stuff fri sxsw marissa mayer googl tim r...,2
...,...,...,...,...,...
8716,Ipad everywhere. #SXSW {link},iPad,Positive emotion,ipad everywher sxsw link,2
8717,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product,wave buzz rt interrupt regular schedul sxsw ge...,1
8718,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product,googl zeiger physician never report potenti ae...,1
8719,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product,verizon iphon custom complain time fell back h...,1


## Trimming the dataframe display

In [430]:
corpus = df[['text', 'target']]

In [431]:
corpus

,text,target
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,0
1,@jessedee Know about @fludapp ? Awesome iPad/i...,2
2,@swonderlin Can not wait for #iPad 2 also. The...,2
3,@sxsw I hope this year's festival isn't as cra...,0
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,2
...,...,...
8716,Ipad everywhere. #SXSW {link},2
8717,"Wave, buzz... RT @mention We interrupt your re...",1
8718,"Google's Zeiger, a physician never reported po...",1
8719,Some Verizon iPhone customers complained their...,1


In [432]:
X = corpus.text
y = corpus.target

In [433]:
X

0       .@wesley83 I have a 3G iPhone. After 3 hrs twe...
1       @jessedee Know about @fludapp ? Awesome iPad/i...
2       @swonderlin Can not wait for #iPad 2 also. The...
3       @sxsw I hope this year's festival isn't as cra...
4       @sxtxstate great stuff on Fri #SXSW: Marissa M...
                              ...                        
8716                        Ipad everywhere. #SXSW {link}
8717    Wave, buzz... RT @mention We interrupt your re...
8718    Google's Zeiger, a physician never reported po...
8719    Some Verizon iPhone customers complained their...
8720    �ϡ�����_��ʋ�΋�ҋ�������⋁_��������_���RT @mentio...
Name: text, Length: 8569, dtype: object

In [434]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 138)

# Vectorizing

In [436]:
c_vec = CountVectorizer(token_pattern = r"(?<![@A-Za-z0-9_])([a-zA-Z]+(?:'[a-z]+)?)",
                        stop_words = sw,
                        preprocessor = prepare_doc)

X_tr_vec = c_vec.fit_transform(X_train)

X_tr_vec = pd.DataFrame.sparse.from_spmatrix(X_tr_vec)

X_tr_vec.columns = sorted(c_vec.vocabulary_)

X_tr_vec.set_index(y_train.index,
                   inplace = True)

X_tr_vec

,aapl,aaron,ab,abacus,abba,aber,abil,abl,abnorm,abound,...,zlf,zms,zomb,zombi,zomg,zone,zoom,zuckerberg,zynga,zzzs
8637,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8060,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5629,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
593,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4030,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Modeling

## Multinomial Naive Bayes

In [437]:
mnb = MultinomialNB()

mnb_cv_one = cross_validate(mnb,
                            X_tr_vec,
                            y_train,
                            cv = 5,
                            scoring = ('accuracy', 'f1_macro'),
                            verbose = 1,
                            n_jobs = -2,
                            return_train_score = True)

cv_printScores(mnb_cv_one)

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.


CV Results
Accuracy
--------------------------------
Training accuracy: 0.818
Test accuracy:     0.647

F-1 Score
--------------------------------
Training F1 score: 0.736
Test F1 score:     0.506


[Parallel(n_jobs=-2)]: Done   5 out of   5 | elapsed:   10.2s finished


**Way** overfit!

## TF-IDF Vectorizer